# **HPC Assignment - 3**

### Name: Anirudh Jakhotia

### Roll No: S20190010007

## **Question: Implement any two sorting algorithms on CUDA Architecture.**

# **Code to setup CUDA on Google Colab**

In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-04-11 17:31:43--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-04-11 17:31:43--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?7UZtVO6PkzUuOHEx9PBNyLEYQqUDMHSWadMyUgy8XTADMNZyZyvOL1ag3BFv60afMYTwWGdWMtYKN0BckjXC_8rH9qH2jYde0EWlb8k0sV1n6

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-931hqsod
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-931hqsod
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=6b9dbe4d62001fc2c63c2d37926ca1156494eb5626add287d14e286c88f4bd15
  Stored in directory: /tmp/pip-ephem-wheel-cache-hy0avzf1/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# Answer: Implementing two sorting algorithms on CUDA Architecture
```
1) Bubble Sort
2) Radix Sort

```



# **Bubble Sort**

In [6]:
%%cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <device_functions.h>
#include <iostream>
#include <stdio.h>
#include <stdlib.h>
#include <chrono>
#include <atomic>
#include <thread>
#include <cuda_profiler_api.h>

using namespace std;

#define SIZE 1024 // FOR PARALEL GPU  SIZE HAS TO BE 2^n 1024
#define THREADS 4 // FOR PARALEL GPU  THREADS = SIZE / (BLOCKS * 2) 
#define BLOCKS 128 //FOR PARALEL GPU  BLOCKS = SIZE / (THREADS * 2) 
string type = "DEVICE";  // USE "HOST" FOR CPU BUBBLE SORT, USE "DEVICE" FOR GPU BUBBLE SORT
int flag = 0;


__host__ void bubbleSortHost(int *array, int index)
{
	int temp;
	do {

		for (int i = 0; i < SIZE - 1 - index * 2 - flag; i++) {
			if (array[index * 2 + i] > array[index * 2 + 1 + i]) {
				temp = array[index * 2 + 1 + i];
				array[index * 2 + 1 + i] = array[index * 2 + i];
				array[index * 2 + i] = temp;
			}
		}

		flag++;

	} while (SIZE - 1 - index * 2 - flag> 0);
}


__global__ void bubbleSortDeviceParallel(int *array, int offSet)
{

	int index = blockIdx.x * blockDim.x + threadIdx.x;
	int indexPerBlock = threadIdx.x;
	int temp;

	if (index  < THREADS* BLOCKS) {

		// FIRST STEP
		if (offSet == 0) {

			// DO THREAD SORTING IN CORRESPONDING BLOCK 
			for (int j = 0; j < THREADS / 2; j++) {

				for (int i = 0; i < THREADS * 2 - 1 - indexPerBlock * 2; i++) {

					if (array[index * 2 + i] > array[index * 2 + 1 + i]) {
						temp = array[index * 2 + 1 + i];
						array[index * 2 + 1 + i] = array[index * 2 + i];
						array[index * 2 + i] = temp;
					}
				}
				__syncthreads();
			}
		}
		
		else {
			if (blockIdx.x != BLOCKS - 1) {
				for (int j = 0; j < THREADS / 2; j++) {
					for (int i = offSet; i < THREADS * 2 - 1 + offSet - indexPerBlock * 2; i++) {

						if (array[index * 2 + i] > array[index * 2 + 1 + i]) {
							temp = array[index * 2 + 1 + i];
							array[index * 2 + 1 + i] = array[index * 2 + i];
							array[index * 2 + i] = temp;
						}

					}
					__syncthreads();
				}
			}
		}
	}

}

int main()
{
	srand(time(NULL));
	int h_count = SIZE;
	int counter = BLOCKS;
	int *h_array;
	int *d_array;
	int offSet;

	h_array = new int[h_count];
	cout << "--------------------------------------" << endl;
    cout << "BUBBLE SORT" << endl;
    cout << "--------------------------------------" << "\n\n";

	//GENERATING RANDOM ELEMENTS FOR h_array
	
	for (int i = 0; i < h_count; i++) {
		h_array[i] = rand() % SIZE;
	}


	// BUBBLE SORT USING CPU
	if (type == "HOST") {

		cout << "ORIGINAL ARRAY: " << endl;
		for (int i = 0; i < SIZE; i++)
		{
			cout << h_array[i] << " ";
		}
		cout << endl;

		cudaEvent_t beginEvent;
		cudaEvent_t endEvent;

		cudaEventCreate(&beginEvent);
		cudaEventCreate(&endEvent);

		cudaEventRecord(beginEvent);

		thread bubbleSortCPU[THREADS];
		for (int i = 0; i < THREADS; i++) {
			bubbleSortCPU[i] = thread(bubbleSortHost, h_array, i);
			bubbleSortCPU[i].join();
		}

		cudaEventRecord(endEvent);
		cudaEventSynchronize(endEvent);

		float timeValue = 0;
		cudaEventElapsedTime(&timeValue, beginEvent, endEvent);

		cout << "CPU Time: " << timeValue << endl;
		cudaEventDestroy(beginEvent);
		cudaEventDestroy(endEvent);

		// ARRAY AFTER BUBBLE SORT
		cout << "BUBBLE SORT RESULTS: " << endl;
		for (int i = 0; i < h_count; i++) {
			cout << h_array[i] << " ";
		}
		cout << endl;
	}

	// BUBBLE SORT USING GPU
	if (type == "DEVICE") {

		cout << "ORIGINAL ARRAY: " << endl;
		for (int i = 0; i < SIZE; i++)
		{
			cout << h_array[i] << " ";
		}
		cout << endl;

		if (cudaMalloc(&d_array, sizeof(int) * h_count) != cudaSuccess)
		{
			cout << "D_ARRAY ALLOCATING NOT WORKING!" << endl;
			return 0;
		}

		if (cudaMemcpy(d_array, h_array, sizeof(int)* h_count, cudaMemcpyHostToDevice) != cudaSuccess)
		{
			cout << "cudaMemcpyHostToDevice ERROR!" << endl;
			cudaFree(d_array);
			return 0;
		}

		cudaEvent_t beginEvent;
		cudaEvent_t endEvent;

		cudaEventCreate(&beginEvent);
		cudaEventCreate(&endEvent);

		cudaEventRecord(beginEvent);

		do {

			for (int i = 0; i < THREADS * 2; i++) {
				offSet = i;
				// POSSIBLE CHANGE: if offset != 0 USE bubbleSortDeviceParallel << < BLOCKS-1, THREADS >> > (d_array, offSet);
				bubbleSortDeviceParallel << < BLOCKS, THREADS >> > (d_array, offSet);
			}

			counter--;
		} while (counter > 0);

		cudaDeviceSynchronize();
		cudaEventRecord(endEvent);
		cudaEventSynchronize(endEvent);

		float timeValue = 0;
		cudaEventElapsedTime(&timeValue, beginEvent, endEvent);

		cout << "GPU Time: " << timeValue << endl;
		cudaEventDestroy(beginEvent);
		cudaEventDestroy(endEvent);

		if (cudaMemcpy(h_array, d_array, sizeof(int)* h_count, cudaMemcpyDeviceToHost) != cudaSuccess)
		{
			delete[] h_array;
			cudaFree(d_array);
			cout << "cudaMemcpyDeviceToHost Error" << endl;
			system("pause");
			return 0;
		}
		cout << endl;

		// ARRAY AFTER BUBBLE SORT
		cout << "SORTED ARRAY: " << endl;
		for (int i = 0; i < h_count; i++) {
			cout << h_array[i] << " ";
		}
		cout << endl;


	}

	// FREEING MEMORY OF CPU & GPU
	delete[] h_array;
	cudaFree(d_array);
	cudaDeviceReset();

	//system("pause");
	return 0;
}

--------------------------------------
BUBBLE SORT
--------------------------------------

ORIGINAL ARRAY: 
917 811 246 726 119 93 576 330 475 852 976 395 625 157 721 520 464 869 962 1014 782 674 828 399 774 283 455 671 231 320 514 124 107 761 851 227 854 403 557 305 231 509 700 856 666 398 353 106 243 291 97 1 966 925 400 716 184 855 363 416 151 878 540 259 615 367 486 445 770 19 750 1002 529 427 834 171 825 163 278 44 455 375 45 397 276 445 89 461 277 452 877 428 306 393 687 921 761 149 342 507 169 69 485 698 496 296 869 297 459 123 341 914 498 386 287 775 831 376 212 84 829 65 513 111 458 176 9 195 326 351 703 495 420 164 169 916 460 14 189 920 138 530 810 636 916 74 387 724 450 599 808 255 664 297 367 99 474 376 294 800 727 997 271 124 138 440 16 598 454 206 494 592 736 281 205 629 355 592 329 805 168 113 37 832 411 404 931 885 780 202 661 483 175 932 607 313 348 624 912 802 830 382 371 542 663 576 147 1018 144 476 800 312 590 837 121 1001 217 28 862 997 230 499 456 406 407 40 719 

# **Radix Sort**

In [7]:
%%cu

#include<iostream>
#include<stdio.h>
using namespace std;
__device__ int function(int value , int bit ,int bitset)
{
    if(bitset == 1 )
    {
        if((value & bit)  != 0)
        {
            return 1;
        }
        else 
            return 0;
    }
    else
    {
        if((value & bit) == 0)
        {
            return 1;
        }
        else 
        {
            return 0;
        }
    }
}
__global__ void predicateDevice(int *d_array , int *d_predicateArrry , int d_numberOfElements,int bit,int bitset)
{
    int index = threadIdx.x + blockIdx.x*blockDim.x;
    if(index < d_numberOfElements)
    {
    
           d_predicateArrry[index] = function(d_array[index],bit,bitset);
    }
}

__global__ void scatter(int *d_array , int *d_scanArray , int *d_predicateArrry,int * d_scatteredArray ,int d_numberOfElements,int offset)
{
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if(index < d_numberOfElements)
    {
        if(d_predicateArrry[index] == 1)
        {
            d_scatteredArray[d_scanArray[index] - 1 +offset ] = d_array[index];
        
        }
    }
}
__global__ void hillisSteeleScanDevice(int *d_array , int numberOfElements, int *d_tmpArray,int moveIndex)
{
    int index = threadIdx.x + blockDim.x * blockIdx.x;
    if(index > numberOfElements)
    {
        return;
    }
    d_tmpArray[index] = d_array[index];
    if(index - moveIndex >=0)
    {
        
        d_tmpArray[index] = d_tmpArray[index] +d_array[index - moveIndex];
    }
}
int* hillisSteeleScanHost(int *d_scanArray,int numberOfElements)
{
    
    
    int *d_tmpArray;
    int *d_tmpArray1;
    cudaMalloc(&d_tmpArray1,sizeof(int)*numberOfElements);
    cudaMalloc(&d_tmpArray,sizeof(int)*numberOfElements);
    cudaMemcpy(d_tmpArray1,d_scanArray,sizeof(int)*numberOfElements,cudaMemcpyDeviceToDevice);
    int j,k=0;
    for(j=1;j<numberOfElements;j= j*2,k++)
    {
        if(k%2 == 0)
        {
            hillisSteeleScanDevice<<<1600,500>>>(d_tmpArray1,numberOfElements,d_tmpArray, j);
            cudaDeviceSynchronize();
        }
        else
        {
            hillisSteeleScanDevice<<<1600,500>>>(d_tmpArray,numberOfElements,d_tmpArray1, j);
            cudaDeviceSynchronize();
        }
    } 
    cudaDeviceSynchronize();
    if(k%2 == 0)
    {
        
        return d_tmpArray1;
    }
    else
    {
        return d_tmpArray;
    }
}
__global__ void print(int *d_predicateArrry,int numberOfElements)
{
    
    for(int i=0;i<numberOfElements;i++)
    {
        printf("index = %d value = %d\n",i,d_predicateArrry[i]);
    }
}

int *compact(int *d_array,int numberOfElements,int bit)
{   
    int offset;
    int *d_predicateArrry;
    cudaMalloc((void**)&d_predicateArrry,sizeof(int)*numberOfElements);
    predicateDevice<<<1600,500>>>(d_array,d_predicateArrry,numberOfElements,bit,0);
    int *d_scanArray;
    d_scanArray = hillisSteeleScanHost(d_predicateArrry,numberOfElements);
    int *d_scatteredArray;
    cudaMalloc((void**)&d_scatteredArray,sizeof(int)*numberOfElements);
    scatter<<<1600,500>>>(d_array,d_scanArray,d_predicateArrry,d_scatteredArray, numberOfElements,0);
    cudaMemcpy(&offset,d_scanArray+numberOfElements-1,sizeof(int),cudaMemcpyDeviceToHost);
    predicateDevice<<<1600,500>>>(d_array,d_predicateArrry,numberOfElements,bit,1);
    d_scanArray = hillisSteeleScanHost(d_predicateArrry,numberOfElements);
    scatter<<<1600,500>>>(d_array,d_scanArray,d_predicateArrry,d_scatteredArray, numberOfElements,offset);
    return d_scatteredArray;
}
int offset;
int *positivenegativesplit(int *d_array,int numberOfElements,int bit,int bitset)
{   
    int *d_predicateArrry;
    cudaMalloc((void**)&d_predicateArrry,sizeof(int)*numberOfElements);
    predicateDevice<<<1600,500>>>(d_array,d_predicateArrry,numberOfElements,bit,bitset);
    int *d_scanArray;
    d_scanArray = hillisSteeleScanHost(d_predicateArrry,numberOfElements);
    int *d_scatteredArray;
    cudaMemcpy(&offset,d_scanArray+numberOfElements-1,sizeof(int),cudaMemcpyDeviceToHost);
    cudaMalloc((void**)&d_scatteredArray,sizeof(int)*offset);
    scatter<<<1600,500>>>(d_array,d_scanArray,d_predicateArrry,d_scatteredArray, numberOfElements,0);
    return d_scatteredArray;
}
int * radixSort(int *d_array , int numberOfElements)
{
    int bit;
    int *d_negativeArray = positivenegativesplit(d_array,numberOfElements,1L<<31,1);
    for(int i=0;i<sizeof(int)*8;i++)
    {
        bit = 1<<i;
        d_negativeArray = compact(d_negativeArray,offset,bit);
    }
    int *d_postiveArray = positivenegativesplit(d_array,numberOfElements,1L<<31,0);
    for(int i=0;i<sizeof(int)*8;i++)
    {
        bit = 1<<i;
        d_postiveArray = compact(d_postiveArray,offset,bit);
    }
    cudaMemcpy(d_array,d_negativeArray,sizeof(int)*(numberOfElements-offset),cudaMemcpyDeviceToDevice);
    cudaMemcpy(d_array+(numberOfElements-offset),d_postiveArray,sizeof(int)*offset,cudaMemcpyDeviceToDevice);
    return d_array;
}
int main()
{
    
    int numberOfElements=300;
    
    int *h_array  = new int[numberOfElements];
    
    //Random Array Generation
    cout << "--------------------------------------" << endl;
    cout << "RADIX SORT" << endl;
    cout << "--------------------------------------" << "\n\n";
    for(int i=0;i<numberOfElements;i++)
    {
        h_array[i]=rand() % 1000;
    }
    cout << "Original Array: " << endl;
    for(int i=0;i<numberOfElements;i++)
    {
       cout <<  h_array[i] << " ";
    }
    cout << "\n\n";
    int *d_array;
    cudaMalloc((void**)&d_array ,sizeof(int)*numberOfElements);
    cudaMemcpy(d_array,h_array,sizeof(int)*numberOfElements,cudaMemcpyHostToDevice);
    d_array = radixSort(d_array, numberOfElements);
    cudaMemcpy(h_array,d_array,sizeof(int)*numberOfElements,cudaMemcpyDeviceToHost);
    cout << "Sorted Array: " << endl;
    for(int i=0;i<numberOfElements;i++)
    {
        cout<<h_array[i]<<" ";
    }
}

--------------------------------------
RADIX SORT
--------------------------------------

Original Array: 
383 886 777 915 793 335 386 492 649 421 362 27 690 59 763 926 540 426 172 736 211 368 567 429 782 530 862 123 67 135 929 802 22 58 69 167 393 456 11 42 229 373 421 919 784 537 198 324 315 370 413 526 91 980 956 873 862 170 996 281 305 925 84 327 336 505 846 729 313 857 124 895 582 545 814 367 434 364 43 750 87 808 276 178 788 584 403 651 754 399 932 60 676 368 739 12 226 586 94 539 795 570 434 378 467 601 97 902 317 492 652 756 301 280 286 441 865 689 444 619 440 729 31 117 97 771 481 675 709 927 567 856 497 353 586 965 306 683 219 624 528 871 732 829 503 19 270 368 708 715 340 149 796 723 618 245 846 451 921 555 379 488 764 228 841 350 193 500 34 764 124 914 987 856 743 491 227 365 859 936 432 551 437 228 275 407 474 121 858 395 29 237 235 793 818 428 143 11 928 529 776 404 443 763 613 538 606 840 904 818 128 688 369 917 917 996 324 743 470 183 490 499 772 725 644 590 505 139 954